In [ ]:
# Loading libraries
library("Matrix")
library(Seurat)
library(DT)
library(dplyr)
library(here)
library(ggplot2)
library(kableExtra)
library(cowplot)
library(tidyverse)
library(reshape)
library(SingleR)
library(ProjecTILs)
library(TILPRED)
library("SingleCellExperiment")
library("AUCell")
#library(ggprism)
library(cowplot)
library(fgsea)
library(org.Mm.eg.db)
library(topGO)
library(annotate)
library("org.Mm.eg.db")
library(biomaRt)
library(furrr)
library(STACAS)

ggtheme <- function() {
  theme(
    axis.text = element_text(size = 20, colour = "black"),
    axis.title = element_text(size = 20, colour = "black"),
    text = element_text(size = 20, colour = "black"),
    legend.text = element_text(size = 20, colour = "black"),
    legend.key.size =  unit(10, units = "points"),
    plot.background = element_blank()
     )
}


In [ ]:
paths  <- list.dirs("../../Project scRNAseq/Analysis of previously published data/069_HPANCDB_Immune_TCRseq/raw/hpapdata/hpapdata/", recursive = F)

In [ ]:
paths  <- list.dirs("../../Project scRNAseq/Analysis of previously published data/069_HPANCDB_Immune_TCRseq/raw/hpapdata/hpapdata/HPAP-001/T cell studies/T cell receptor repertoire TCRb Vregion seq/HPAP-001_TCRbulk_Spleen_DNA_100ng_Replicate-1.clones_TRB.tsv", recursive = F)

In [ ]:
paths  <- paste0(paths, "/T cell studies/T cell receptor repertoire TCRb Vregion seq/")

In [ ]:
paths

In [ ]:
get_data  <- function(path){
    file_path  <- list.files(path)
    file_path1  <- file_path[grepl(file_path, pattern = "Replicate-1.clones_TRB.tsv")]
    file_path2  <- file_path[grepl(file_path, pattern = "Replicate-2.clones_TRB.tsv")]
    
    dat1  <- fread(file_path1)
    dat2  <- fread(file_path2)
}

In [ ]:
library(data.table)

In [ ]:
get_mixcr_tcr_data  <- function(path){
    file_path  <- list.files(path, full.names = T)
    file_name  <- list.files(path)
    
    file_path1  <- file_path[grepl(file_path, pattern = "Replicate-1.clones_TRB.tsv")]
    file_path2  <- file_path[grepl(file_path, pattern = "Replicate-2.clones_TRB.tsv")]
    
    dat1  <- fread(file_path1)
    dat2  <- fread(file_path2)

    dat1  <- dat1  %>% filter(minQualCDR3 == 58)
    dat2  <- dat2  %>% filter(minQualCDR3 == 58)
    dat1$Patient_ID  <- substr(file_name[1],1,8)
    dat1$Replicate  <- "Rep1"

    dat2$Patient_ID  <- substr(file_name[1],1,8)
    dat2$Replicate  <- "Rep2"

    dat  <- rbind(dat1, dat2)
    return(dat)
}


In [ ]:
all_tcrs  <- map(.x = paths, .f = get_mixcr_tcr_data)

In [ ]:
fct_qual_as_char  <- function(df){
    df$minQualCDR2  <- as.character(df$minQualCDR2)
    df$minQualFR3  <- as.character(df$minQualFR3)
    return(df)
}

In [ ]:
all_tcrs2  <- map(.x = all_tcrs, .f = fct_qual_as_char)

In [ ]:
all_tcrs3  <- all_tcrs2  %>% bind_rows

In [ ]:
all_tcrs3  %>% group_by(Patient_ID)  %>% tally  %>% arrange(n)

In [ ]:
write.csv(all_tcrs3, "hpap_tcr.csv")

In [ ]:
all_tcrs3  <- read_csv("hpap_tcr.csv")

In [ ]:
all_tcrs3

In [ ]:
patient_metadata  <- read_delim("../../Project scRNAseq/Analysis of previously published data/069_HPANCDB_Immune_TCRseq/hpap_medatata.csv")

In [ ]:
patient_metadata

In [ ]:
patient_metadata  %>% filter(donor_ID %in% all_tcrs3$Patient_ID)  %>% pull(clinical_diagnosis)  %>% table

In [ ]:
all_tcrs4  <- left_join(all_tcrs3, patient_metadata  %>% mutate(Patient_ID = donor_ID))

In [ ]:
all_tcrs4

In [ ]:
colnames(all_tcrs4)

## CDR3 beta length

In [ ]:
metadata_6  <- all_tcrs4

In [ ]:
metadata_6$cdr3_B_nchar  <- nchar(metadata_6$aaSeqCDR3)

In [ ]:
metadata_6$Sample_ID  <- paste(metadata_6$Patient_ID, metadata_6$Replicate)

In [ ]:
options(repr.plot.width = 17)
metadata_6 %>% 
filter(as.numeric(cdr3_B_nchar) <23)  %>% 
filter(clinical_diagnosis %in% c("T1D control", "T1DM"))  %>% 
mutate(Disease = ifelse(grepl(pattern = "control", x = clinical_diagnosis), "Ctrl", "Dia"))  %>% 
group_by(Disease, Sample_ID, cdr3_B_nchar)  %>% 
tally  %>% 
filter(!is.na(cdr3_B_nchar))  %>% 
ggplot(aes(x = Disease, y = n)) + 
  geom_boxplot(outlier.shape = NA, aes(fill = Disease), alpha = 0.4) + 
  geom_dotplot(binaxis='y', stackdir='center', dotsize=0) +
  geom_jitter(width = 0.2, height = 0.5, size = 0.5, alpha = 1, aes(color = Disease)) +  # in aes, you can also use shape or fill (for the shapes that allow it)
  facet_wrap(~factor(cdr3_B_nchar),
             scales = "fixed", ncol = 19, drop = T) +
 ylab("Log2 clonal abundance") +
  xlab("CDR3 beta length") +
  theme_classic() + 
  ggpubr::stat_compare_means(label.x.npc = 0.3, label = "p.format") + 
theme(strip.background = element_blank(), axis.text.x = element_blank(), 
     axis.ticks.x = element_blank())

In [ ]:
options(repr.plot.width = 17)
metadata_6 %>% 
filter(as.numeric(cdr3_B_nchar) <23)  %>% 
filter(clinical_diagnosis %in% c("T1D control", "T1DM"))  %>% 
mutate(Disease = ifelse(grepl(pattern = "control", x = clinical_diagnosis), "Ctrl", "Dia"))  %>% 
group_by(Disease, Sample_ID, cdr3_B_nchar)  %>% 
tally  %>% 
filter(!is.na(cdr3_B_nchar))  %>% 
ggplot(aes(x = Disease, y = n)) + 
  geom_boxplot(outlier.shape = NA, aes(fill = Disease), alpha = 0.4) + 
  geom_dotplot(binaxis='y', stackdir='center', dotsize=0) +
  geom_jitter(width = 0.2, height = 0.5, size = 0.5, alpha = 1, aes(color = Disease)) +  # in aes, you can also use shape or fill (for the shapes that allow it)
  facet_wrap(~factor(cdr3_B_nchar),
             scales = "fixed", ncol = 19, drop = T) +
 ylab("Log2 clonal abundance") +
  xlab("CDR3 beta length") +
  theme_classic() + 
  ggpubr::stat_compare_means(label.x.npc = 0.3, label = "p.format") + 
theme(strip.background = element_blank(), axis.text.x = element_blank(), 
     axis.ticks.x = element_blank())

- každou sekvenci u jednoho pacienta počítat tolikrát, kolikrát tam je
- počítám počet dané délky pro jednoho pacienta
- potom plotuji frekvenci dané délky u daného pacienta

In [ ]:
options(repr.plot.width = 17)
test_length  <- metadata_6 %>% 
filter(as.numeric(cdr3_B_nchar) <23)  %>% 
filter(clinical_diagnosis %in% c("T1D control", "T1DM"))  %>% 
mutate(Disease = ifelse(grepl(pattern = "control", x = clinical_diagnosis), "Ctrl", "Dia"))  %>% 
group_by(Disease, Patient_ID, cdr3_B_nchar)  %>%
  summarise(n = n()) %>%
  mutate(freq = n / sum(n))

In [ ]:
test_length  %>% 
filter(!is.na(cdr3_B_nchar))  %>% 
ggplot(aes(x = Disease, y = freq)) + 
  geom_boxplot(outlier.shape = NA, aes(fill = Disease), alpha = 0.4) + 
  geom_dotplot(binaxis='y', stackdir='center', dotsize=0) +
  geom_jitter(width = 0.2, height = 0, size = 0.5, alpha = 1, aes(color = Disease)) +  # in aes, you can also use shape or fill (for the shapes that allow it)
  facet_wrap(~factor(cdr3_B_nchar),
             scales = "fixed", ncol = 19, drop = T) +
 ylab("Frequency") +
  xlab("CDR3 beta length") +
  theme_classic() + 
  ggpubr::stat_compare_means(label.x.npc = 0.3, label = "p.format") + 
theme(strip.background = element_blank(), axis.text.x = element_blank(), 
     axis.ticks.x = element_blank())

- každou sekvenci u jednoho pacienta počítat pouze jednou
- počítám počet dané délky pro jednoho pacienta
- potom plotuji frekvenci dané délky u daného pacienta

In [ ]:
all_tcrs4

## Plot for figure

In [ ]:
options(repr.plot.width = 17)
metadata_6 %>% 
filter(as.numeric(cdr3_B_nchar) <23)  %>% 
filter(clinical_diagnosis %in% c("T1D control", "T1DM"))  %>% 
mutate(Disease = ifelse(grepl(pattern = "control", x = clinical_diagnosis), "Ctrl", "Dia"))  %>% 
group_by(Disease, Sample_ID, cdr3_B_nchar)  %>% 
tally  %>% 
filter(!is.na(cdr3_B_nchar))  %>% 
ggplot(aes(x = Disease, y = n)) + 
  geom_boxplot(outlier.shape = NA, aes(fill = Disease), alpha = 0.4) + 
  geom_dotplot(binaxis='y', stackdir='center', dotsize=0) +
  geom_jitter(width = 0.2, height = 0.5, size = 0.5, alpha = 1, aes(color = Disease)) +  # in aes, you can also use shape or fill (for the shapes that allow it)
  facet_wrap(~factor(cdr3_B_nchar),
             scales = "fixed", ncol = 19, drop = T) +
 ylab("Log2 clonal abundance") +
  xlab("CDR3 beta length") +
  theme_classic() + 
  ggpubr::stat_compare_means(label.x.npc = 0.3, label = "p.format") + 
theme(strip.background = element_blank(), axis.text.x = element_blank(), 
     axis.ticks.x = element_blank())

In [ ]:
metadata_6  %>% colnames

In [ ]:
options(repr.plot.width = 17)
test_length  <- metadata_6 %>% 
dplyr::select(clinical_diagnosis, Patient_ID, cdr3_B_nchar, aaSeqCDR3)  %>% 
unique  %>% 
filter(clinical_diagnosis %in% c("T1D control", "T1DM"))  %>% 
mutate(Disease = ifelse(grepl(pattern = "control", x = clinical_diagnosis), "Ctrl", "Dia"))  %>% 
group_by(Disease, Patient_ID, cdr3_B_nchar)  %>%
  summarise(n = n()) %>%
  mutate(freq = n / sum(n))

test_length  %>% group_by(Patient_ID)  %>% summarise(sum = sum(freq))

In [ ]:
test_length  %>% 
filter(!is.na(cdr3_B_nchar))  %>% 
filter(as.numeric(cdr3_B_nchar) <26)  %>% 
ggplot(aes(x = Disease, y = freq)) + 
  geom_boxplot(outlier.shape = NA, aes(fill = Disease), alpha = 0.4) + 
  geom_dotplot(binaxis='y', stackdir='center', dotsize=0) +
  geom_jitter(width = 0.2, height = 0, size = 0.5, alpha = 1, aes(color = Disease)) +  # in aes, you can also use shape or fill (for the shapes that allow it)
  facet_wrap(~factor(cdr3_B_nchar),
             scales = "fixed", ncol = 21, drop = T) +
 ylab("Frequency") +
  xlab("CDR3 beta length") +
  theme_classic() + 
  ggpubr::stat_compare_means(label.x.npc = 0.3, label = "p.format") + 
theme(strip.background = element_blank(), axis.text.x = element_blank(), 
     axis.ticks.x = element_blank())

In [ ]:
test_length  %>% 
filter(!is.na(cdr3_B_nchar))  %>% 
filter(as.numeric(cdr3_B_nchar) <24)  %>% 
ggplot(aes(x = Disease, y = freq)) + 
  geom_boxplot(outlier.shape = NA, aes(fill = Disease), alpha = 0.4) + 
  geom_dotplot(binaxis='y', stackdir='center', dotsize=0) +
  geom_jitter(width = 0.2, height = 0, size = 0.7, shape = 21, alpha = 0.7, aes(fill = Disease)) +  # in aes, you can also use shape or fill (for the shapes that allow it)
  facet_wrap(~factor(cdr3_B_nchar),
             scales = "fixed", ncol = 21, drop = T) +
 scale_fill_manual(values = c("dodgerblue", "red2"))  +
ylab("Frequency") +
  xlab("CDR3 beta length") +
  theme_classic() + 
  ggpubr::stat_compare_means(label.x.npc = 0.3, label = "p.format", size = 7) + 
theme(strip.background = element_blank(), axis.text.x = element_blank(), 
     axis.ticks.x = element_blank()) + ggtheme()
ggsave(filename = "../240218_VN_Diabetes_V05/figures/tcr_plots/tcr_length/hpap.png", width = 22, height = 10, units = "cm")
ggsave(filename = "../240218_VN_Diabetes_V05/figures/tcr_plots/tcr_length/hpap.svg", width = 22, height = 10, units = "cm")

# Change metadata to our format

In [ ]:
source("../240218_VN_Diabetes_V05/code/diabetes_analysis_v05.R")

In [ ]:
colnames(all_tcrs4)

     'orig.ident' 'nCount_RNA' 'nFeature_RNA' 'percent.mt' 'percent.rp' 'nCount_Antibodies' 'nFeature_Antibodies' 'hashtags' 'Experiment_ID' 'Date_prepared' 'Prepared_by' 'Sample_ID' 'Well' 'Hashtag' 'Hashtag_sequence' 'Hashtag_cat_n' 'Hashtag_dilution' 'Project.code' 'Well_CT_ID' 'Condition' 'Disease' 'Time' 'Species' 'Age' 'Age_group' 'Sex' 'Library_code' 'Patient_ID' 'is_cell' 'chain' 'contig_id' 'high_confidence' 'productive' 'length' 'reads' 'umis' 'vdj_source' 'count_TRA' 'count_TRB' 'count_Multi' 'cdr3_A1' 'cdr3_A1_nt' 'cdr3_A2' 'cdr3_A2_nt' 'cdr3_B' 'cdr3_B_nt' 'cdr3_Multi' 'cdr3_Multi_nt' 'v_gene_A1' 'j_gene_A1' 'v_gene_A2' 'j_gene_A2' 'v_gene_B' 'd_gene_B' 'j_gene_B' 'v_gene_Multi' 'd_gene_Multi' 'j_gene_Multi' 'clonotypes' 'has.VDJ' 'has.complete.VDJ' 'nCount_PTPRC' 'nFeature_PTPRC' 'RNA_snn_res.0.5' 'seurat_clusters' 'clonotype.repeated' 'RNA_snn_res.0.8' 'Patient_Time' 'Sample_char' 'patient' 'rc' 'dat_nar' 'time_taken' 'age_taken' 'age_group' 'sex' 'dat_man' 'age_man' 'hba1c_man' 'ph_man' 'ahco3_man' 'random_cpeptide_man' 'bh_man' 'bh_man_sds' 'bw_man' 'bw_man_sds' 'bmi_man' 'bmi_man_sds' 'gad_man' 'iaa_man' 'ia2_man' 'znt8_man' 'date_taken_1' 'fasting_cpept_1' 'wbc' 'lymph_abs' 'mon_abs' 'neutr_abs' 'eoz_abs' 'baz_abs' 'lymph_perc' 'mon_perc' 'neutr_perc' 'eoz_perc' 'baz_perc' 'tdd_1' 'tdd_per_kg_1' 'fasting_cpept_T1' 'idaa1c' 'part_remission_y_n' 'c_peptide_change' 'c_peptide_change_perc' 'HLA_A1' 'HLA_A2' 'HLA_B1' 'HLA_B2' 'HLA_C1' 'HLA_C2' 'HLA_DMA1' 'HLA_DMA2' 'HLA_DMB1' 'HLA_DMB2' 'HLA_DOA1' 'HLA_DOA2' 'HLA_DPA11' 'HLA_DPA12' 'HLA_DPB11' 'HLA_DPB12' 'HLA_DQA11' 'HLA_DQA12' 'HLA_DQB11' 'HLA_DQB12' 'HLA_DRA1' 'HLA_DRA2' 'HLA_DRB11' 'HLA_DRB12' 'HLA_E1' 'HLA_E2' 'HLA_F1' 'HLA_F2' 'sample' 'pct_CD8_of_live' 'pct_naive_of_CD8' 'pct_eff_of_CD8' 'pct_CD4_of_live' 'pct_naive_of_CD4' 'pct_eff_of_CD4' 'sort_ratio_naive_to_eff_CD8' 'sort_ratio_naive_to_eff_CD4' 'is_old_control' 'RNA_snn_res.1' 'barcode' 'Monaco_single' 'HPCA_single' 'Wherry_main' 'nCount_refAssay' 'nFeature_refAssay' 'predicted.celltype.l1.score' 'predicted.celltype.l1' 'predicted.celltype.l2.score' 'predicted.celltype.l2' 'predicted.celltype.l3.score' 'predicted.celltype.l3' 'mapping.score' 'RNA_snn_res.0.6' 'group_cpept' 'Condition2' 'integrated_snn_res.0.6' 'annotations_manual' 'nCount_pathwayswmean' 'nFeature_pathwayswmean' 'nCount_CollecTRI' 'nFeature_CollecTRI' 'nCount_prediction.score.celltype.l1' 'nFeature_prediction.score.celltype.l1' 'nCount_prediction.score.celltype.l2' 'nFeature_prediction.score.celltype.l2' 'nCount_prediction.score.celltype.l3' 'nFeature_prediction.score.celltype.l3' 'clone_nt' 'clone_aa' 'test' 'clone_abundance' 'log_clone_abundance' 'clone_abundance_group' 'cdr3_B_nchar' 'cdr3_B_nchar_nucl' 'clone_aa_noNA' 



In [ ]:
all_tcrs4

In [ ]:
all_tcrs5  <- all_tcrs4  %>% 
mutate(cdr3_B = aaSeqCDR3,
      Sample_char = paste(Patient_ID, clinical_diagnosis))  %>% 
filter(clinical_diagnosis %in% c("T1DM", "T1D control"))

In [ ]:
all_tcrs5

In [ ]:
clone_table_individual <- all_tcrs5 %>%
  dplyr::group_by(cdr3_B, Sample_char) %>%
          dplyr::summarize(n = n()) %>%
    arrange(desc(n)) %>% pivot_wider(names_from = Sample_char, values_from = n, values_fill = 0) 

In [ ]:
clone_table_individual

In [ ]:
is_positive <- function(number){
  number2 <- ifelse(is.na(number),0,ifelse(number==0,0,1))
  return(number2)
}

In [ ]:
df_all4  <- get_df_all4_for_tcr_analysis(clone_table_individual)

In [ ]:
dir.create("../figures/tcr/", recursive = T)

In [ ]:
df_all4

In [ ]:
  df24 <- df_all4
    df24[df24 == 1] <- 0

    matrix_4  <- as.matrix(df24)
    

In [ ]:
    sample_annot <- data.frame(row.names = rownames(matrix_4), 
                         rn = rownames(matrix_4))  %>% 
                separate(rn, sep = " ", 
                         remove = F,
                         into = c("Patient_ID","Disease"))  

In [ ]:
sample_annot

In [ ]:
sample_annot  <- sample_annot   %>%  mutate(Disease = ifelse(Disease == "T1D", "Ctrl", "T1DM"))

In [ ]:
sample_annot

In [ ]:
        pheatmap::pheatmap(matrix_4, 
                           cluster_rows = T, 
                           cluster_cols = T, 
                           #filename = paste0("../figures/tcr/",sample_name,"_heatmap.png"), 
                       width = 17, 
                       height = 17)

In [ ]:
overlap_index  <- df_all4  %>% 
    rownames_to_column("var1")  %>% 
    pivot_longer(!var1, names_to = "var2", values_to = "overlap")  %>% 
    unique  %>% as.data.frame()  %>% 
separate(var1, sep = " ", remove = F, into = c("Patient_ID","Disease_1"))  %>% 
    separate(var2, sep = " ", remove = F, into = c("Patient_ID","Disease_2"))  %>% 
    mutate(comparison_type = ifelse(
    var1 == var2, "SELF - SELF", ifelse(
    Disease_1 == "T1DM" & Disease_2 == "T1DM", "DIA - DIA", ifelse(
    Disease_1 == "T1D" & Disease_2 == "T1D", "CTRL - CTRL",   "DIA - CTRL"
    ))))

In [ ]:
overlap_index 

In [ ]:
    overlap_index %>% 
    filter(comparison_type != "SELF - SELF")  %>% 
    ggplot(aes(x = comparison_type, y = overlap)) +  
    geom_dotplot(binaxis='y', stackdir='center', dotsize=0) + 
      ggrastr::rasterize(geom_jitter(position=position_jitter(0.2), size = 1, color = "grey70", alpha = 0.1)) +
       geom_violin(aes(color = comparison_type), scale = "width", alpha = 0.7) +  theme_classic() + 

       NoLegend() + theme(axis.text.x = element_text(angle = 45, vjust = 0.8, hjust =0.8)) +
      ggtitle("Overlap") + 
      xlab("Compared diagnoses") +
      ylab("Percentage of shared") 

In [ ]:
    ggsave(paste0("../figures/tcr/",sample, "_overlap1.png"), width = 15, height = 11, units = "cm")
    ggsave(paste0("../figures/tcr/",sample, "_overlap1.svg"), width = 15, height = 11, units = "cm")

In [ ]:
    overlap_index %>% 
    filter(comparison_type != "SELF - SELF")  %>% 
    filter(comparison_type != "SELF T1 - SELF T0")  %>% 
    ggplot(aes(x = comparison_type, y = overlap)) +  
    geom_dotplot(binaxis='y', stackdir='center', dotsize=0) + 
      ggrastr::rasterize(geom_jitter(position=position_jitter(0.2), size = 1, color = "grey70", alpha = 0.1)) +
       geom_violin(aes(color = comparison_type), scale = "width", alpha = 0.7) +  theme_classic() + 

       NoLegend() + theme(axis.text.x = element_text(angle = 45, vjust = 0.8, hjust =0.8)) +
      ggtitle("Overlap without self self") + 
ggpubr::stat_compare_means(comparisons = list( c(1,3), c(2,3), c(1,2)), size = 7)+
    ggpubr::stat_compare_means(size = 7, label = "p.format") +
    
      xlab("Compared diagnoses") +
      ylab("Percentage of shared") 

In [ ]:
    ggsave(paste0("../figures/tcr/",sample, "_overlap2.png"), width = 15, height = 11, units = "cm")
    ggsave(paste0("../figures/tcr/",sample, "_overlap2.svg"), width = 15, height = 11, units = "cm")

In [ ]:
    overlap_index %>% 
    filter(comparison_type %in% c("CTRL - CTRL", 
                                  "DIA - CTRL",
                                  "DIA - DIA",
                                  "SELF T1 - SELF T0"))  %>% 
    ggplot(aes(x = comparison_type, y = log(overlap+0.001))) +  
    geom_dotplot(binaxis='y', stackdir='center', dotsize=0) + 
      ggrastr::rasterize(geom_jitter(position=position_jitter(0.2), size = 1, color = "grey70", alpha = 0.7)) +
       geom_violin(aes(color = comparison_type), scale = "width", alpha = 0.7) +  theme_classic() + 

       NoLegend() + theme(axis.text.x = element_text(angle = 45, vjust = 0.8, hjust =0.8)) +
    scale_color_manual(values = c("blue", "purple", "red", "grey20")) + 
    stat_summary(fun = "median",
                   geom = "crossbar", 
                   width = 0.5,
                   colour = "grey44") + 
   ggpubr::stat_compare_means(comparisons = list( c(1,3), c(2,3), c(1,2)), size = 7)+
    ggpubr::stat_compare_means(size = 7, label = "p.format") +
    ggtheme() +
    theme(axis.ticks.x = element_blank()) +
      xlab("Compared diagnoses") +
      ylab("log(percentage of shared)") 

In [ ]:
    ggsave(paste0("../figures/tcr/",sample, "_overlap_final.png"), width = 11, height = 18, units = "cm")
    ggsave(paste0("../figures/tcr/",sample, "_overlap_final.svg"), width = 11, height = 18, units = "cm")

## Published clones

In [ ]:
published_clones  <- read_excel("..//240218_VN_Diabetes_V05/data/published_clones.xlsx")

In [ ]:
published_trb  <- published_clones$CDR3b[which(!is.na(published_clones$CDR3b))]

### TRB

In [ ]:
all_tcrs5

In [ ]:
table(all_tcrs5$cdr3_B %in% published_trb)

In [ ]:
published_clones  %>% filter(CDR3b %in% all_tcrs5$cdr3_B)  %>% filter(!is.na(CDR3b))

In [ ]:
table(cd8_l2_subcluster$cdr3_B %in% published_trb)

In [ ]:
published_clones  %>% filter(CDR3b %in% cd8_l2_subcluster$cdr3_B)  %>% filter(!is.na(CDR3b))

In [ ]:
cd8_l2_subcluster@meta.data  %>% 
dplyr::select(Patient_ID, Condition, Experiment_ID,cdr3_B, cdr3_B)  %>% 
filter(cdr3_B %in% published_trb)

HLA of our patients:

### Kmers

In [ ]:
library(kebabs)

In [ ]:
published_trb2  <- published_clones$CDR3b[which(!is.na(published_clones$CDR3b))]

In [ ]:
published_trb2  <- published_clones  %>% filter(!is.na(CDR3b))  %>% pull(CDR3b)

In [ ]:
published_trb2

In [ ]:
kebabs::AAVector(published_trb)

In [ ]:
s1 <- kebabs::AAVector(published_trb)

In [ ]:
sk14 <- spectrumKernel(k=4, normalized=T)

In [ ]:
kmers_trb  <- as.data.frame(colSums(as.data.frame(drop(getExRep(s1, sk14)))))

In [ ]:
kmers_trb

In [ ]:
kmers_trb$kmer  <- rownames(kmers_trb)

In [ ]:
colnames(kmers_trb)  <- c("ref", "kmer")

In [ ]:
rownames(kmers_trb)  <- NULL

In [ ]:
kmers_trb

In [ ]:
metadata_y  <- all_tcrs5

In [ ]:
df_all  <- kmers_trb

for(i in 1:length(levels(factor(metadata_y$Patient_ID)))){
    pt1_betas  <- metadata_y %>%
    filter(Patient_ID == levels(factor(metadata_y$Patient_ID))[i])  %>% 
      dplyr::group_by(cdr3_B) %>%
          dplyr::summarize(n = n()) %>%
    arrange(desc(n))
    
    rep_pt  <- rep(pt1_betas$cdr3_B[1:nrow(pt1_betas)], pt1_betas$n[1:nrow(pt1_betas)])
    
    s1 <- kebabs::AAVector(rep_pt)
    sk14 <- spectrumKernel(k=4, normalized=TRUE)  
  
    kmers_trb  <- as.data.frame(colSums(as.data.frame(drop(getExRep(s1, sk14)))))
    kmers_trb$kmer  <- rownames(kmers_trb)
    colnames(kmers_trb)  <- c(levels(factor(metadata_y$Patient_ID))[i], "kmer")
    rownames(kmers_trb)  <- NULL
    
    df_all  <- left_join(df_all, kmers_trb)
    
}

In [ ]:
df_all

In [ ]:
df_all[is.na(df_all)] <- 0

In [ ]:
df_all$ref  <- NULL

In [ ]:
colnames(all_tcrs5)

In [ ]:
md  <- all_tcrs5  %>% group_by(Patient_ID, clinical_diagnosis)  %>% tally()  %>% dplyr::select(-n)

In [ ]:
df_all2  <- df_all  %>% pivot_longer(!kmer, names_to = "Patient_ID", values_to = "freq")   %>% 
left_join(md)

In [ ]:
df_all2

In [ ]:
m_w  <- data.frame(kmer = "kmer", pvalue = "pvalue")

In [ ]:
dim(df_all2)

In [ ]:
nrow(md)

In [ ]:
length(levels(factor(df_all2$kmer)))

In [ ]:
for(i in 1:24531){
    
    df3  <- df_all2  %>% filter(kmer == levels(factor(df_all2$kmer))[i] & clinical_diagnosis %in% c("T1D control","T1DM"))
    df3$clinical_diagnosis  <- factor(df3$clinical_diagnosis, c("T1D control","T1DM"))
    df4  <- data.frame(kmer = levels(factor(df_all2$kmer))[i], 
                       pvalue = wilcox.test(freq~clinical_diagnosis, data = df3)$p.value)
    m_w  <- rbind(m_w, df4)
    
}

In [ ]:
m_w2 <- m_w[2:nrow(m_w),] 

In [ ]:
m_w  %>% arrange(pvalue)

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 16)
df_all2  %>% filter(kmer %in% (m_w  %>% arrange(pvalue))$kmer[1:24])  %>% 
ggplot(aes(x = clinical_diagnosis, y = freq)) + # you can change the x to whatever variable you're interested in
  geom_boxplot(outlier.shape = NA, 
           alpha = 0.7, width = 0.9, aes(fill = clinical_diagnosis)) +
   geom_dotplot(binaxis='y', stackdir='center', dotsize=0) + 
   geom_jitter(binaxis='y', position=position_jitter(width = 0.1, height = 0), 
  size = 3, shape = 21, stackdir='center', aes(fill = clinical_diagnosis), color = "black") + 
  facet_wrap(~kmer, scales = "free", ncol = 6) +
  ylab("Frequency") +
  xlab("Condition") +
  theme_classic() +
 ylim(0,NA) +
  theme(strip.background = element_blank(), panel.grid = element_blank()) 

In [ ]:
signif_kmers  <- m_w  %>% dplyr::filter(pvalue < 0.05)  %>% pull(kmer)

In [ ]:
write.csv(signif_kmers, "signif_kmers_HPAP.csv")

In [ ]:
m_w  %>% dplyr::filter(kmer == "SPTT")

In [ ]:
options(repr.plot.width = 5, repr.plot.height = 4)
df_all2  %>% filter(kmer == "SPTT")  %>% 
ggplot(aes(x = clinical_diagnosis, y = freq)) + # you can change the x to whatever variable you're interested in
  geom_boxplot(outlier.shape = NA, 
           alpha = 0.7, width = 0.9, aes(fill = clinical_diagnosis)) +
   geom_dotplot(binaxis='y', stackdir='center', dotsize=0) + 
   geom_jitter(binaxis='y', position=position_jitter(width = 0.1, height = 0), 
  size = 3, shape = 21, stackdir='center', aes(fill = clinical_diagnosis), color = "black") + 
  facet_wrap(~kmer, scales = "free", ncol = 6) +
  ylab("Frequency") +
  xlab("Condition") +
  theme_classic() +
 ylim(0,NA) +
  theme(strip.background = element_blank(), panel.grid = element_blank()) 

In [ ]:
m_w  %>% dplyr::filter(kmer == "GGPT")

In [ ]:
options(repr.plot.width = 5, repr.plot.height = 4)
df_all2  %>% filter(kmer == "GGGS")  %>% 
ggplot(aes(x = clinical_diagnosis, y = freq)) + # you can change the x to whatever variable you're interested in
  geom_boxplot(outlier.shape = NA, 
           alpha = 0.7, width = 0.9, aes(fill = clinical_diagnosis)) +
   geom_dotplot(binaxis='y', stackdir='center', dotsize=0) + 
   geom_jitter(binaxis='y', position=position_jitter(width = 0.1, height = 0), 
  size = 3, shape = 21, stackdir='center', aes(fill = clinical_diagnosis), color = "black") + 
  facet_wrap(~kmer, scales = "free", ncol = 6) +
  ylab("Frequency") +
  xlab("Condition") +
  theme_classic() +
 ylim(0,NA) +
  theme(strip.background = element_blank(), panel.grid = element_blank()) 

In [ ]:
m_w  %>% dplyr::filter(kmer == "LAAN")

options(repr.plot.width = 5, repr.plot.height = 4)
df_all2  %>% filter(kmer == "LAAN")  %>% 
ggplot(aes(x = clinical_diagnosis, y = freq)) + # you can change the x to whatever variable you're interested in
  geom_boxplot(outlier.shape = NA, 
           alpha = 0.7, width = 0.9, aes(fill = clinical_diagnosis)) +
   geom_dotplot(binaxis='y', stackdir='center', dotsize=0) + 
   geom_jitter(binaxis='y', position=position_jitter(width = 0.1, height = 0), 
  size = 3, shape = 21, stackdir='center', aes(fill = clinical_diagnosis), color = "black") + 
  facet_wrap(~kmer, scales = "free", ncol = 6) +
  ylab("Frequency") +
  xlab("Condition") +
  theme_classic() +
 ylim(0,NA) +
  theme(strip.background = element_blank(), panel.grid = element_blank()) 

In [ ]:
options(repr.plot.width = 15, repr.plot.height = 4)
df_all2  %>% filter(kmer %in% c("GGPP","HEQY","SLRR","SPTT"))  %>% 
ggplot(aes(x = factor(clinical_diagnosis, labels = c("Ctrl","T1DM")), y = freq)) + # you can change the x to whatever variable you're interested in
  geom_boxplot(outlier.shape = NA, 
           alpha = 0.7, width = 0.9, aes(fill = clinical_diagnosis)) +
   geom_dotplot(binaxis='y', stackdir='center', dotsize=0) + 
   geom_jitter(binaxis='y', position=position_jitter(width = 0.1, height = 0), 
  size = 3, shape = 21, stackdir='center', aes(fill = clinical_diagnosis), color = "black") + 
  facet_wrap(~kmer, scales = "free", ncol = 6) +
scale_fill_manual(values = c("dodgerblue","indianred"))+
  ylab("Frequency") +
  xlab("Condition") +
ggpubr::stat_compare_means(size = 7, label = "p.format", label.x = 1.3,
                              label.y.npc = 0.9) +
  theme_classic() + ggtheme() +
 ylim(0,NA) +
  theme(strip.background = element_blank(), panel.grid = element_blank()) 

In [ ]:
options(repr.plot.width = 7, repr.plot.height = 4)
df_all2  %>% filter(kmer %in% c("PGTG"))  %>% 
ggplot(aes(x = factor(clinical_diagnosis, labels = c("Ctrl","T1DM")), y = freq)) + # you can change the x to whatever variable you're interested in
  geom_boxplot(outlier.shape = NA, 
           alpha = 0.7, width = 0.9, aes(fill = clinical_diagnosis)) +
   geom_dotplot(binaxis='y', stackdir='center', dotsize=0) + 
   geom_jitter(binaxis='y', position=position_jitter(width = 0.1, height = 0), 
  size = 3, shape = 21, stackdir='center', aes(fill = clinical_diagnosis), color = "black") + 
  facet_wrap(~kmer, scales = "free", ncol = 6) +
scale_fill_manual(values = c("dodgerblue","indianred"))+
  ylab("Frequency") +
  xlab("Condition") +
ggpubr::stat_compare_means(size = 7, label = "p.format", label.x = 1.3,
                              label.y.npc = 0.9) +
  theme_classic() + ggtheme() +
 ylim(0,NA) +
  theme(strip.background = element_blank(), panel.grid = element_blank()) 

## Peptides

# TCR properties peptides

In [ ]:
all_tcrs5$cdr3_B

In [ ]:
library(Peptides)
library(Seurat)
library(dplyr)
    
add_TCR_properties  <- function(df_tcr){
    df_tcr1  <- df_tcr  %>% mutate(pI_cdr3_B = ifelse(is.na(cdr3_B),NA_integer_,pI(cdr3_B)),
           boman_cdr3_B = ifelse(is.na(cdr3_B),NA_integer_,boman(cdr3_B)),
          charge_cdr3_B = ifelse(is.na(cdr3_B),NA_integer_,charge(cdr3_B)),
          hmoment_cdr3_B = ifelse(is.na(cdr3_B),NA_integer_,hmoment(cdr3_B)),
          hydrophobicity_cdr3_B = ifelse(is.na(cdr3_B),NA_integer_,hydrophobicity(cdr3_B)),
          mw_cdr3_B = ifelse(is.na(cdr3_B),NA_integer_,mw(cdr3_B)),
          mz_cdr3_B = ifelse(is.na(cdr3_B),NA_integer_,mz(cdr3_B)),
          )
    return(df_tcr1)
    }

In [ ]:
library(ggpubr)

In [ ]:
all_tcrs4_tcr  <- add_TCR_properties(all_tcrs5)

In [ ]:
colnames(all_tcrs4_tcr)[131:137]

## All clones

In [ ]:
dir.create("figures_tcr_peptides_HPAP")

In [ ]:
options(repr.plot.width = 4, repr.plot.height = 6.5)


for(i in 131:137){
   df  <- data.frame(Score = all_tcrs4_tcr[,i],
                  Annotation = all_tcrs4_tcr$clinical_diagnosis) 
    colnames(df)[1]  <- "Score"
    dataMedian <- summarise(group_by(df, Annotation), MD = 1.2*median(Score, na.rm = T))
       
p  <- ggplot(df, aes(x = Annotation, y = Score)) +
    ggrastr::rasterise(geom_jitter(position=position_jitter(0.2), size = 1, color = "grey70", alpha = 0.7)) +
    geom_violin(aes(color = Annotation), scale = "width", alpha = 0.7) +  theme_classic() + 
   NoLegend() + theme(axis.text.x = element_text(angle = 45, vjust = 0.8, hjust =0.8)) +
    scale_color_manual(values = c("blue", "#c41515ff")) + 
    stat_summary(fun = "median",
                   geom = "crossbar", 
                   width = 0.6,
                   colour = "grey20") +
    geom_text(data = dataMedian, aes(Annotation, MD, label = round(MD, digits = 2)), 
             size = 7) +
    xlab("") +
    scale_fill_continuous(guide=FALSE) +
   #scale_y_continuous(limits = c(NA, max())) +
   ggpubr::stat_compare_means(size = 7, label = "p.format", label.x = 1.3,
                              label.y.npc = 0.9) +
  ggtitle(colnames(all_tcrs4_tcr)[i]) + 
  theme(panel.background = element_blank(), axis.text.x = element_text(angle = 90),
      axis.ticks.x = element_blank()) + ggtheme()
   print(p)
    ggsave(paste0("figures_tcr_peptides_HPAP/",colnames(all_tcrs4_tcr)[i], ".svg"), width = 7, height = 14, units = "cm")
    ggsave(paste0("figures_tcr_peptides_HPAP/",colnames(all_tcrs4_tcr)[i], ".png"), width = 7, height = 14, units = "cm")
}

In [ ]:
library(ggbeeswarm)

## All clones by patient

In [ ]:

for(i in 131:137){
   df  <- data.frame(Score = all_tcrs4_tcr[,i],
                  Disease = all_tcrs4_tcr$clinical_diagnosis,
                    Patient_ID = all_tcrs4_tcr$Patient_ID
                 ) 
    colnames(df)[1]  <- "Score"
    dataMedian <- summarise(group_by(df, Disease), MD = 1.2*median(Score, na.rm = T))
       
p  <-    df  %>% 
    group_by(Patient_ID, Disease)  %>% 
    summarise(avg_score = mean(Score, na.rm = TRUE))  %>% 
    ggplot(aes(x = Disease, y = avg_score)) + # you can change the x to whatever variable you're interested in
   geom_violin(alpha = 0.3, aes(fill = Disease), scale = "width") + 
stat_summary(fun = "median",
               geom = "crossbar", 
               width = 0.75,
               color = "grey30") +
geom_beeswarm(size = 3, aes(fill = Disease), cex = 3, 
                shape = 21, color = "black", corral = "random") +
scale_fill_manual(values = c("#1874cdff",   "#c41515ff","#eeb4b4ff")) +
scale_color_manual(values = c("dodgerblue3",   "#aa2a2aff","#e88989ff")) + #ggpubr::stat_compare_means(comparisons = list( c(1,3), c(2,3), c(1,2)), size = 7)+
    ggpubr::stat_compare_means(size = 7, label = "p.format") +
  ggtitle(colnames(all_tcrs4_tcr)[i]) + 
  theme(panel.background = element_blank()) + ggtheme()
   print(p)
    ggsave(paste0("figures_tcr_peptides_HPAP/",colnames(all_tcrs4_tcr)[i], "_bypatient.svg"), width = 7, height = 14, units = "cm")
    ggsave(paste0("figures_tcr_peptides_HPAP/",colnames(all_tcrs4_tcr)[i], "_bypatient.png"), width = 7, height = 14, units = "cm")
}

## One clone counted just once

In [ ]:
one_random_clone  <- function(df_all_tcr){
 
df_tcr_oneclone  <- df_all_tcr  %>% group_by(cdr3_B, Patient_ID)  %>% slice_sample(n = 1)

    return(df_tcr_oneclone)
    }

In [ ]:
all_tcrs4_tcr_one_random  <- one_random_clone(all_tcrs4_tcr)

In [ ]:

for(i in 131:137){
   df  <- data.frame(Score = all_tcrs4_tcr_one_random[,i],
                  Annotation = all_tcrs4_tcr_one_random$clinical_diagnosis) 
    colnames(df)[1]  <- "Score"
    dataMedian <- summarise(group_by(df, Annotation), MD = 1.2*median(Score, na.rm = T))
       
p  <- ggplot(df, aes(x = Annotation, y = Score)) +
    ggrastr::rasterise(geom_jitter(position=position_jitter(0.2), size = 1, color = "grey70", alpha = 0.7)) +
    geom_violin(aes(color = Annotation), scale = "width", alpha = 0.7) +  theme_classic() + 
   NoLegend() + theme(axis.text.x = element_text(angle = 45, vjust = 0.8, hjust =0.8)) +
    scale_color_manual(values = c("blue", "#c41515ff")) + 
    stat_summary(fun = "median",
                   geom = "crossbar", 
                   width = 0.6,
                   colour = "grey20") +
    geom_text(data = dataMedian, aes(Annotation, MD, label = round(MD, digits = 2)), 
             size = 7) +
    xlab("") +
    scale_fill_continuous(guide=FALSE) +
   #scale_y_continuous(limits = c(NA, max())) +
   ggpubr::stat_compare_means(size = 7, label = "p.format", label.x = 1.3,
                              label.y.npc = 0.9) +
  ggtitle(colnames(all_tcrs4_tcr_one_random)[i]) + 
  theme(panel.background = element_blank(), axis.text.x = element_text(angle = 90),
      axis.ticks.x = element_blank()) + ggtheme()
   print(p)
    ggsave(paste0("figures_tcr_peptides_HPAP/",colnames(all_tcrs4_tcr)[i], "_one_random.svg"), width = 7, height = 14, units = "cm")
    ggsave(paste0("figures_tcr_peptides_HPAP/",colnames(all_tcrs4_tcr)[i], "_one_random.png"), width = 7, height = 14, units = "cm")
}

## One clone counted just once by patient

In [ ]:
dir.create("../figures/tcr/peptides_cd4_one_random_bypatient/")

In [ ]:

for(i in 131:137){
   df  <- data.frame(Score = all_tcrs4_tcr_one_random[,i],
                  Disease = all_tcrs4_tcr_one_random$clinical_diagnosis,
                    Patient_ID = all_tcrs4_tcr_one_random$Patient_ID
                 ) 
    colnames(df)[1]  <- "Score"
    dataMedian <- summarise(group_by(df, Disease), MD = 1.2*median(Score, na.rm = T))
       
p  <-    df  %>% 
    group_by(Patient_ID, Disease)  %>% 
    summarise(avg_score = mean(Score, na.rm = TRUE))  %>% 
    ggplot(aes(x = Disease, y = avg_score)) + # you can change the x to whatever variable you're interested in
   geom_violin(alpha = 0.3, aes(fill = Disease), scale = "width") + 
stat_summary(fun = "median",
               geom = "crossbar", 
               width = 0.75,
               color = "grey30") +
geom_beeswarm(size = 3, aes(fill = Disease), cex = 3, 
                shape = 21, color = "black", corral = "random") +
scale_fill_manual(values = c("#1874cdff",   "#c41515ff","#eeb4b4ff")) +
scale_color_manual(values = c("dodgerblue3",   "#aa2a2aff","#e88989ff")) + #ggpubr::stat_compare_means(comparisons = list( c(1,3), c(2,3), c(1,2)), size = 7)+
    ggpubr::stat_compare_means(size = 7, label = "p.format") +
  ggtitle(colnames(all_tcrs4_tcr_one_random)[i]) + 
  theme(panel.background = element_blank()) + ggtheme()
   print(p)
    ggsave(paste0("figures_tcr_peptides_HPAP/",colnames(all_tcrs4_tcr)[i], "_bypatient_onerandom.svg"), width = 7, height = 14, units = "cm")
    ggsave(paste0("figures_tcr_peptides_HPAP/",colnames(all_tcrs4_tcr)[i], "_bypatient_onerandom.png"), width = 7, height = 14, units = "cm")
}

## TCR properties table

In [ ]:
get_tcr_prop_table  <- function(i) {
    
    # All clones
     df  <- data.frame(Score = all_tcrs4_tcr[,i],
                  Annotation = all_tcrs4_tcr$clinical_diagnosis) 
    colnames(df)[1]  <- "Score"
    df  <- df  %>% dplyr::filter(!is.na(Score))
    
    wcx  <- wilcox.test(df$Score ~ df$Annotation, conf.int = T)

    df2  <- df  %>% group_by(Annotation)  %>% summarise(mean = mean(Score), sd = sd(Score))

    df_all  <- data.frame(name = colnames(all_tcrs4_tcr)[i], 
                          cell_type = "HPAP",
                          test_type = "All clones",
                          estimate = wcx$estimate,
                          pval = wcx$p.value, 
                          mean_dia = df2$mean[2],
                         mean_ctrl = df2$mean[1], 
                         upper = wcx$conf.int[2],
                         lower = wcx$conf.int[1]
                         ) 
    
     df_final  <-  df_all
    
    # Random clones
    df  <- data.frame(Score = all_tcrs4_tcr_one_random[,i],
                  Annotation = all_tcrs4_tcr_one_random$clinical_diagnosis)  
     colnames(df)[1]  <- "Score"
    df  <- df  %>% dplyr::filter(!is.na(Score))
    
    
    wcx  <- wilcox.test(df$Score ~ df$Annotation, conf.int = T)

    df2  <- df  %>% group_by(Annotation)  %>% summarise(mean = mean(Score), sd = sd(Score))

     df_all  <- data.frame(name = colnames(all_tcrs4_tcr_one_random)[i], 
                          cell_type = "HPAP",
                          test_type = "Random clones",
                          estimate = wcx$estimate,
                          pval = wcx$p.value, 
                          mean_dia = df2$mean[2],
                         mean_ctrl = df2$mean[1], 
                         upper = wcx$conf.int[2],
                         lower = wcx$conf.int[1]
                         ) 
    
        df_final  <- rbind(df_final, df_all)
    
    
    # All clones by patient
    
      df  <- data.frame(Score = all_tcrs4_tcr[,i],
                  Disease = all_tcrs4_tcr$clinical_diagnosis,
                    Patient_ID = all_tcrs4_tcr$Patient_ID
                 ) 
    colnames(df)[1]  <- "Score" 
    
    df  <- df  %>% 
    dplyr::filter(!is.na(Score))  %>% 
    group_by(Patient_ID, Disease)  %>% 
    summarise(Score = mean(Score)) 
    
    wcx  <- wilcox.test(df$Score ~ df$Disease, conf.int = T)

    df2  <- df  %>% group_by(Disease)  %>% summarise(mean = mean(Score), sd = sd(Score))

     df_all  <- data.frame(name = colnames(all_tcrs4_tcr)[i], 
                          cell_type = "HPAP",
                          test_type = "All clones by patient",
                          estimate = wcx$estimate,
                          pval = wcx$p.value, 
                          mean_dia = df2$mean[2],
                         mean_ctrl = df2$mean[1], 
                         upper = wcx$conf.int[2],
                         lower = wcx$conf.int[1]
                         ) 
    
        df_final  <- rbind(df_final, df_all)
    
    # Random clones by patient
    
      df  <- data.frame(Score = all_tcrs4_tcr_one_random[,i],
                  Disease = all_tcrs4_tcr_one_random$clinical_diagnosis,
                    Patient_ID = all_tcrs4_tcr_one_random$Patient_ID
                 ) 
    colnames(df)[1]  <- "Score" 
    
    df  <- df  %>% 
    dplyr::filter(!is.na(Score))  %>% 
    group_by(Patient_ID, Disease)  %>% 
    summarise(Score = mean(Score)) 
    
    wcx  <- wilcox.test(df$Score ~ df$Disease, conf.int = T)

    df2  <- df  %>% group_by(Disease)  %>% summarise(mean = mean(Score), sd = sd(Score))

     df_all  <- data.frame(name = colnames(all_tcrs4_tcr_one_random)[i], 
                          cell_type = "HPAP",
                          test_type = "Random clones by patient",
                          estimate = wcx$estimate,
                          pval = wcx$p.value, 
                          mean_dia = df2$mean[2],
                         mean_ctrl = df2$mean[1], 
                         upper = wcx$conf.int[2],
                         lower = wcx$conf.int[1]
                         ) 
    
        df_final  <- rbind(df_final, df_all)
    return(df_final)
}

In [ ]:
for(i in 131:137)    {
        # All clones
     df  <- data.frame(Score = all_tcrs4_tcr[,i],
                  Annotation = all_tcrs4_tcr$clinical_diagnosis) 
    colnames(df)[1]  <- "Score"
    df  <- df  %>% dplyr::filter(!is.na(Score))
    
    wcx  <- wilcox.test(df$Score ~ df$Annotation, conf.int = T)

    df2  <- df  %>% group_by(Annotation)  %>% summarise(mean = mean(Score), sd = sd(Score))

    df_all  <- data.frame(name = colnames(all_tcrs4_tcr)[i], 
                          cell_type = "HPAP",
                          test_type = "All clones",
                          estimate = wcx$estimate,
                          pval = wcx$p.value, 
                          mean_dia = df2$mean[2],
                         mean_ctrl = df2$mean[1], 
                         upper = wcx$conf.int[2],
                         lower = wcx$conf.int[1]
                         ) 
    
     df_final  <-  df_all
    
    # Random clones
    df  <- data.frame(Score = all_tcrs4_tcr_one_random[,i],
                  Annotation = all_tcrs4_tcr_one_random$clinical_diagnosis)  
     colnames(df)[1]  <- "Score"
    df  <- df  %>% dplyr::filter(!is.na(Score))
    
    
    wcx  <- wilcox.test(df$Score ~ df$Annotation, conf.int = T)

    df2  <- df  %>% group_by(Annotation)  %>% summarise(mean = mean(Score), sd = sd(Score))

     df_all  <- data.frame(name = colnames(all_tcrs4_tcr_one_random)[i], 
                          cell_type = "HPAP",
                          test_type = "Random clones",
                          estimate = wcx$estimate,
                          pval = wcx$p.value, 
                          mean_dia = df2$mean[2],
                         mean_ctrl = df2$mean[1], 
                         upper = wcx$conf.int[2],
                         lower = wcx$conf.int[1]
                         ) 
    
        df_final  <- rbind(df_final, df_all)
    
    
    # All clones by patient
    
      df  <- data.frame(Score = all_tcrs4_tcr[,i],
                  Disease = all_tcrs4_tcr$clinical_diagnosis,
                    Patient_ID = all_tcrs4_tcr$Patient_ID
                 ) 
    colnames(df)[1]  <- "Score" 
    
    df  <- df  %>% 
    dplyr::filter(!is.na(Score))  %>% 
    group_by(Patient_ID, Disease)  %>% 
    summarise(Score = mean(Score)) 
    
    wcx  <- wilcox.test(df$Score ~ df$Disease, conf.int = T)

    df2  <- df  %>% group_by(Disease)  %>% summarise(mean = mean(Score), sd = sd(Score))

     df_all  <- data.frame(name = colnames(all_tcrs4_tcr)[i], 
                          cell_type = "HPAP",
                          test_type = "All clones by patient",
                          estimate = wcx$estimate,
                          pval = wcx$p.value, 
                          mean_dia = df2$mean[2],
                         mean_ctrl = df2$mean[1], 
                         upper = wcx$conf.int[2],
                         lower = wcx$conf.int[1]
                         ) 
    
        df_final  <- rbind(df_final, df_all)
    
    # Random clones by patient
    
      df  <- data.frame(Score = all_tcrs4_tcr_one_random[,i],
                  Disease = all_tcrs4_tcr_one_random$clinical_diagnosis,
                    Patient_ID = all_tcrs4_tcr_one_random$Patient_ID
                 ) 
    colnames(df)[1]  <- "Score" 
    
    df  <- df  %>% 
    dplyr::filter(!is.na(Score))  %>% 
    group_by(Patient_ID, Disease)  %>% 
    summarise(Score = mean(Score)) 
    
    wcx  <- wilcox.test(df$Score ~ df$Disease, conf.int = T)

    df2  <- df  %>% group_by(Disease)  %>% summarise(mean = mean(Score), sd = sd(Score))

     df_all  <- data.frame(name = colnames(all_tcrs4_tcr_one_random)[i], 
                          cell_type = "HPAP",
                          test_type = "Random clones by patient",
                          estimate = wcx$estimate,
                          pval = wcx$p.value, 
                          mean_dia = df2$mean[2],
                         mean_ctrl = df2$mean[1], 
                         upper = wcx$conf.int[2],
                         lower = wcx$conf.int[1]
                         ) 
    
        df_final  <- rbind(df_final, df_all)
    
    if(i>131){
        df_final_final  <- rbind(df_final_final, df_final)
    } else {
        df_final_final  <- df_final
    }
}

In [ ]:
df_final_final

In [ ]:
write.csv(df_final_final, "../240218_VN_Diabetes_V05/tables/tcr/peptides/hpap_all_props.csv")

In [ ]:
cd4_props  <- read_csv("../240218_VN_Diabetes_V05/tables/tcr/peptides/cd4_tcr_all_props_table.csv")
cd8_props  <- read_csv("../240218_VN_Diabetes_V05/tables/tcr/peptides/cd8_tcr_all_props_table.csv")
hpap_props  <- read_csv("../240218_VN_Diabetes_V05/tables/tcr/peptides/hpap_all_props.csv")

In [ ]:
all_props  <- rbind(cd8_props, cd4_props, hpap_props)

In [ ]:
all_props$test  <- "test"

In [ ]:
install.packages("ggh4x")

In [ ]:
options(repr.plot.width = 9, repr.plot.height = 20)

all_props  %>% 
mutate(cell_type_test_type = paste(cell_type, test_type))  %>% 
dplyr::filter(grepl(name, pattern = "cdr3_B"))  %>% 
ggplot(aes(x = cell_type_test_type, y = estimate), color = 'grey') +
# geom_vline(xintercept = 0, color = "gray75") +
 geom_hline(yintercept = 0, color = "grey") +
  geom_point(aes(color = pval<0.05), size = 2) +
  theme_classic() +
  scale_color_manual(values = c("grey","indianred3"))+
  facet_wrap(~name, scales = "free", ncol = 1) +
#scale_shape_manual(values=c("\u25BA","\u25C4")) +
#ggnewscale::new_scale_fill() +
geom_linerange(aes(ymin = lower, ymax = upper, color = pval<0.05),
                 alpha = 0.5) +
#theme(axis.text.x = element_text(angle = 90)) +
coord_flip()
# ggh4x::facet_nested(test ~ name + test_type, scales = "free_y", independent = "y") 
 #ggh4x::facet_grid2(rows = vars(test_type), cols = vars(name), scales = "free_y", independent = "y")

In [ ]:
options(repr.plot.width = 9, repr.plot.height = 20)

all_props  %>% 
mutate(cell_type_test_type = paste(cell_type, test_type))  %>% 
dplyr::filter(grepl(name, pattern = "cdr3_B"))  %>% 
ggplot(aes(x = cell_type_test_type, y = estimate), color = 'grey') +
# geom_vline(xintercept = 0, color = "gray75") +
 geom_hline(yintercept = 0, color = "grey") +
  geom_point(aes(color = pval<0.05), size = 2) +
  theme_classic() +
  scale_color_manual(values = c("grey","indianred3"))+
  facet_wrap(~name, scales = "free", ncol = 1) +
#scale_shape_manual(values=c("\u25BA","\u25C4")) +
#ggnewscale::new_scale_fill() +
geom_linerange(aes(ymin = lower, ymax = upper, color = pval<0.05),
                 alpha = 0.5) +
#theme(axis.text.x = element_text(angle = 90)) +
coord_flip()
# ggh4x::facet_nested(test ~ name + test_type, scales = "free_y", independent = "y") 
 #ggh4x::facet_grid2(rows = vars(test_type), cols = vars(name), scales = "free_y", independent = "y")

In [ ]:
all_props$name  %>% table

In [ ]:
options(repr.plot.width = 9, repr.plot.height = 20)

all_props  %>% 
mutate(cell_type_test_type = paste(cell_type, test_type))  %>% 
dplyr::filter(grepl(name, pattern = "cdr3_A1"))  %>% 
ggplot(aes(x = cell_type_test_type, y = estimate), color = 'grey') +
# geom_vline(xintercept = 0, color = "gray75") +
 geom_hline(yintercept = 0, color = "grey") +
  geom_point(aes(color = pval<0.05), size = 2) +
  theme_classic() +
  scale_color_manual(values = c("grey","indianred3"))+
  facet_wrap(~name, scales = "free", ncol = 1) +
#scale_shape_manual(values=c("\u25BA","\u25C4")) +
#ggnewscale::new_scale_fill() +
geom_linerange(aes(ymin = lower, ymax = upper, color = pval<0.05),
                 alpha = 0.5) +
#theme(axis.text.x = element_text(angle = 90)) +
coord_flip()
# ggh4x::facet_nested(test ~ name + test_type, scales = "free_y", independent = "y") 
 #ggh4x::facet_grid2(rows = vars(test_type), cols = vars(name), scales = "free_y", independent = "y")

In [ ]:
options(repr.plot.width = 9, repr.plot.height = 20)

all_props  %>% 
mutate(cell_type_test_type = paste(cell_type, test_type))  %>% 
dplyr::filter(grepl(name, pattern = "cdr3_clone"))  %>% 
ggplot(aes(x = cell_type_test_type, y = estimate), color = 'grey') +
# geom_vline(xintercept = 0, color = "gray75") +
 geom_hline(yintercept = 0, color = "grey") +
  geom_point(aes(color = pval<0.05), size = 2) +
  theme_classic() +
  scale_color_manual(values = c("grey","indianred3"))+
  facet_wrap(~name, scales = "free", ncol = 1) +
#scale_shape_manual(values=c("\u25BA","\u25C4")) +
#ggnewscale::new_scale_fill() +
geom_linerange(aes(ymin = lower, ymax = upper, color = pval<0.05),
                 alpha = 0.5) +
#theme(axis.text.x = element_text(angle = 90)) +
coord_flip()
# ggh4x::facet_nested(test ~ name + test_type, scales = "free_y", independent = "y") 
 #ggh4x::facet_grid2(rows = vars(test_type), cols = vars(name), scales = "free_y", independent = "y")

In [ ]:
all_props$name  %>% table

In [ ]:
all_props  %>% 
filter(name %in% c("boman_cdr3_B","hmoment_cdr3_B","hydrophobicity_cdr3_B"))   %>% 
head

In [ ]:
ggtheme <- function() {
  theme(
    axis.text = element_text(size = 20),
    axis.title = element_text(size = 20),
    text = element_text(size = 20, colour = "black"),
    legend.text = element_text(size = 20),
    legend.key.size =  unit(10, units = "points")
    
  )
}


In [ ]:
options(repr.plot.width = 9, repr.plot.height = 6)

all_props  %>% 
filter(name %in% c("boman_cdr3_B","hmoment_cdr3_B","hydrophobicity_cdr3_B"))  %>% 
mutate(cell_type_test_type = paste(cell_type, test_type))  %>% 
dplyr::filter(test_type %in% c("All clones"))  %>% 
ggplot(aes(x = cell_type_test_type, y = estimate), color = 'grey') +
# geom_vline(xintercept = 0, color = "gray75") +
 geom_hline(yintercept = 0, color = "grey", linewidth = 1) +
  geom_point(aes(color = pval<0.05), size = 4) +
  theme_classic() +
  scale_color_manual(values = c("grey","indianred3"))+
  facet_wrap(~name, scales = "fixed", ncol = 1) +
#scale_shape_manual(values=c("\u25BA","\u25C4")) +
#ggnewscale::new_scale_fill() +
geom_linerange(aes(ymin = lower, ymax = upper, color = pval<0.05),
                 alpha = 0.5, linewidth = 2) + ggtheme() +
#theme(axis.text.x = element_text(angle = 90)) +
coord_flip()  + xlab("")
ggsave("../240617_VN_Diabetes_V06/figures/tcr_peptides/peptides.svg", create.dir = T, 
      width = 20, height = 15, units = "cm")

In [ ]:
tcr_all_props_table2  <- bind_rows(tcr_all_props_table)

In [ ]:
tcr_all_props_table2  %>% arrange(pval)

In [ ]:
dir.create( "../tables/tcr/peptides/")

In [ ]:
write.csv(tcr_all_props_table2, "../tables/tcr/peptides/cd4_tcr_all_props_table.csv")